In [80]:
--1.  Find the Total number of Unique Transactions, Senders and Recipients with Successful transactions ?

SELECT count(distinct payment_id) as num_payments,
count(distinct sender_id) as num_senders,
count(distinct recipient_id) as num_recipients,
from 'payments_data.csv' 
where payment_state = 'SUCCESS'


,num_payments,num_senders,num_recipients
0,257,152,171


In [81]:
--2. Find the failure rate of transactions by sender's geographical state, sort the results in descending order of failure rate?

SELECT state, round(avg(case when payment_state='FAILURE' THEN 1 else 0 end),2) as failure_rate
from 'payments_data.csv' p 
left join 'customers_data.csv' c
on p.sender_id = c.customer_id
group by 1
order by 2 desc

,state,failure_rate
0,MN,0.90
1,MA,0.89
2,MO,0.83
3,WV,0.80
4,CT,0.78
5,MT,0.75
6,KY,0.75
7,AL,0.67
8,GA,0.67
9,OK,0.63


In [79]:
--Question 3. Find the Customer(s) that sent or received the most amount in the year 2022?

with t1 as (
(SELECT customer_id, sum(amount_cents) as total_amount
from 'payments_data.csv' p 
left join 'customers_data.csv' c
on p.sender_id = c.customer_id
WHERE extract(year from p.created_at) = 2022
group by 1
)
union

(SELECT customer_id, sum(amount_cents) as total_amount
from 'payments_data.csv' p 
left join 'customers_data.csv' c
on p.recipient_id = c.customer_id
WHERE extract(year from p.created_at) = 2022
group by 1
)
	), t2 as (
select customer_id, sum(total_amount) as total_amount 
from t1
group by 1
order by 2 desc 
)
Select customer_id, total_amount
From t2 
Where total_amount = (select max(total_amount) from t2)

,customer_id,total_amount
0,774205,238676.0


In [77]:
--4. Find all Customers that opened an account in August 2022 that have sent or received >$100 in the first 30 days of opening the account?

with t1 as (
(----Total Sent	
SELECT customer_id, sum(amount_cents) as total_amount
from 'payments_data.csv' p 
left join 'customers_data.csv' c
on p.sender_id = c.customer_id
and (p.created_at - c.created_at) <= 30
WHERE extract(year from c.created_at) = 2022
and extract(month from c.created_at) = 8
and payment_state = 'SUCCESS'
group by 1
)
union
(----Total Received	
SELECT customer_id, sum(amount_cents) as total_amount
from 'payments_data.csv' p 
left join 'customers_data.csv' c
on p.recipient_id = c.customer_id
and (p.created_at - c.created_at) <= 30
WHERE extract(year from c.created_at) = 2022
and extract(month from c.created_at) = 8
and payment_state = 'SUCCESS'
group by 1
)
	)
select customer_id, sum(total_amount) as total_amount_sent_received 
from t1
group by 1
having (sum(total_amount)/100) > 100
order by 2 desc 


,customer_id,total_amount_sent_received
0,829271,87089.0
1,716684,68101.0


In [78]:
--5. Find pairs of customers who sent and received money between each other ?

SELECT distinct p1.sender_id,p1.recipient_id
from 'payments_data.csv' p1 join 'payments_data.csv' p2 
on p1.sender_id = p2.recipient_id 
and p1.recipient_id = p2.sender_id
and p1.payment_state = 'SUCCESS' 
AND p2.payment_state = 'SUCCESS'
order by 1


,sender_id,recipient_id
0,160092,219345
1,204607,801918
2,219345,160092
3,616489,803476
4,801918,204607
5,803476,616489
